# Tutorial 14: Params Classes

This tutorial walks you through the use of different parameters classes. Understanding how these classes work is fundamental in designing your own experiments in Flow. 

The classes featured in this tutorial are `InitialConfig`,`SimParams`, `SumoParams`, `SumoCarFollowingParams`, `EnvParams`, `VehicleParams`, and `SumoLaneChangeParams`. Each of these classes is detailed in their corresponding sections in this tutorial. All of them are defined in __flow/core/params.py__.

This tutorial uses the classical Ring experiment involving a loop with a single lane and a fixed number of vehicles on it, similar to what is done in tutorial 1. Unlike tutorial 1, this one focuses on the use of the parameters in the classes.

**This tutorial is divided into 7 sections and is organized as follows:**

1. `InitialConfig`
2. `SimParams`
3. `SumoParams`
4. `SumoCarFollowingParams`
5. `EnvParams`
6. `VehicleParams`
7. `SumoLaneChangeParams`


## 1. InitialConfig

`InitialConfig` specifies parameters that affect the positioning of vehicles in the network at the start of a rollout. These parameters can be used, for instance, to limit the edges and number of lanes vehicles initially occupy or to provide a means of adding randomness to the starting positions of vehicles. By default, the vehicles are uniformly distributed in the network, across all edges and lanes.

The "vehicles" mean the vehicles that are initially present in the network when the simulation starts, i.e. not the vehicles added during the simulation via inflows but those that were added via `vehicles.add(num_vehicles=...)`.

### 1.1 List of possible parameters

Following are the parameters which can be set up within `InitialConfig`, along with their default values and description:


- `shuffle` (_bool, optional_): specifies whether the ordering of the vehicles should be shuffled. Default to False.
- `spacing` (_str, optional_): specifies the initial positioning of the vehicles. Can be either one of: 
    - "uniform": (default value): the vehicles will be placed uniformly
    - "random": the vehicles will be placed randomly
    - "custom": you must specify a custom ordering
- `min_gap` (_float, optional_): the minimum gap to be kept between two vehicles among all the vehicles, in meters. Defaults to 0 meters. 
- `x0` (_float, optional_): position of the first vehicle to be placed in the network. Can be used to shift the position of all the vehicles. The position is an edge position (c.f. the `specify_edge_starts` method in the scenario). Defaults to 0.
- `perturbation` (_float, optional_): standard deviation used to perturb the vehicles from their uniform position, in meters (this is only used in case `spacing` is set to "uniform"). Defaults to 0 meters.
- `bunching` (_float, optional_): reduce the portion of the network that should be filled with vehicles by this amount, in edge length. Defaults to 0. 
- `lanes_distribution` (_int, optional_): number of lanes the vehicles should be dispersed into. If the value is greater than the total number of lanes on an edge, vehicles are spread across all lanes. Defaults to infinity.
- `edges_distribution` (_str / str list / (str -> int) dict, optional_): edges vehicles may be placed on. Can be either one of:
    - "all": vehicles are distributed over all edges (default)
    - list of edges vehicles can be distributed over
    - dict where for each item, the key is the name of the edge and the value is the number of cars to place on that edge.
- `additional_params` (_dict, optional_): some other network-specific parameters.

### 1.2 Example using the Ring experiment

The following lines of codes create a basic experiment in a ring, with 22 vehicles whose initial respective position of each one can modified using `InitialConfig`. If you don't understand the following code, head over to tutorial 1 which explains it.

Let's start by creating the vehicles that will be affected by the initial configuration, as well as the configuration itself:

In [1]:
from flow.core.params import VehicleParams, InitialConfig
from flow.controllers import IDMController, ContinuousRouter

# we add 22 vehicles that will be affected by the InitialConfig object
vehicles = VehicleParams()
vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)

# set up initial config 
initial_config = InitialConfig(
    spacing="uniform",
    bunching=20
)

Then, let's write the rest of the code necessary for the simulation, and run it.

In [2]:
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, NetParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

sim_params = SumoParams(sim_step=0.1, render=True)
net_params = NetParams(additional_params=ADDITIONAL_NET_PARAMS)
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)
exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 438.98462690001725
Average, std return: 438.98462690001725, 0.0
Average, std speed: 2.9266271862486435, 0.0


{'returns': [438.98462690001725],
 'velocities': [array([0.08577842, 0.17039769, 0.2538258 , ..., 2.9723844 , 2.97238436,
         2.97238432])],
 'mean_returns': [0.2926564179333444],
 'per_step_returns': [[0.008577841752034304,
   0.01703976566300353,
   0.025382572549459154,
   0.033603306012689835,
   0.041699252949377284,
   0.05014762423043607,
   0.05842187387272349,
   0.06611145348018753,
   0.0736674828825087,
   0.08108846436721634,
   0.08837313152369042,
   0.09552044441662831,
   0.1025295840672158,
   0.1093999462936479,
   0.11613113496788281,
   0.12272295475022006,
   0.12917540336729352,
   0.13548866350213284,
   0.14166309436691366,
   0.14769922302979357,
   0.1535977355667045,
   0.15935946810718535,
   0.16498539784030591,
   0.17047663404258478,
   0.17583440918468088,
   0.181060070167756,
   0.1861550697339544,
   0.19112095808870133,
   0.19595937476567316,
   0.20067204075860762,
   0.20526075093775867,
   0.20972736676296208,
   0.21407380930006784,
   0.2

Feel free to play around with the parameters set in `InitialConfig`. For example, you can try the following:

In [3]:
initial_config = InitialConfig(
    spacing="uniform",
    perturbation=5
)

or, for instance,

In [4]:
initial_config = InitialConfig(
    spacing="random"
)

## 2. SimParams

These are the simulation-specific parameters.
Following is how the class is defined in `params.py`.

        def __init__(self,
                 sim_step=0.1,
                 render=False,
                 restart_instance=False,
                 emission_path=None,
                 save_render=False,
                 sight_radius=25,
                 show_radius=False,
                 pxpm=2,
                 color_vehicles=True):
        """Instantiate SimParams."""
        self.sim_step = sim_step
        self.render = render
        self.restart_instance = restart_instance
        self.emission_path = emission_path
        self.save_render = save_render
        self.sight_radius = sight_radius
        self.pxpm = pxpm
        self.show_radius = show_radius
        self.color_vehicles = color_vehicles
        

### 2.1 List of possible parameters

- `sim_step` (_float, optional_): seconds per simulation step; 0.1 by default
- `render` (_str or bool, optional_): specifies whether to visualize the rollout(s)
     - False: no rendering
     - True: delegate rendering to sumo-gui for back-compatibility
     - "gray": static grayscale rendering, which is good for training
     - "dgray": dynamic grayscale rendering
     - "rgb": static RGB rendering
     - "drgb": dynamic RGB rendering, which is good for visualization
- `restart_instance` (_bool, optional_): specifies whether to restart a simulation upon reset. Restarting the instance helps avoid slowdowns cause by excessive inflows over large experiment runtimes, but also require the gui to be started after every reset if "render" is set to True.
- `emission_path` (_str, optional_): Path to the folder in which to create the emissions output. Emissions output is not generated if this value is not specified
- `save_render` (_bool, optional_): specifies whether to save rendering data to disk
- `sight_radius` (_int, optional_): sets the radius of observation for RL vehicles (meter)
- `show_radius` (_bool, optional_): specifies whether to render the radius of RL observation
- `pxpm` (_int, optional_): specifies rendering resolution (pixel / meter)
- `color_vehicles` (_bool, optional_): whether or not to automatically color vehicles according to their types



### 2.2 Extensions

We did not show an example under this once since the parameters above can be associated to either one of the 2 extensions of this class which are:

1. SumoParams (_This is discussed in the next section._)
2. AimsunParam (_This includes parameters which can be used for running a simulation in Aimsun, thus it is better to refer to __Tutorial 2 - Aimsun__)

## 3. SumoParams

This is an extension of the class `SimParams` where it allows the users to have SUMO-specific simulation parameters.

These parameters are used to customize a SUMO simulation instance upon initialization. This includes passing the simulation step length, specifying whether to use SUMO's gui during a run, and other features described in the parameters below with their corresponding default values.
    
    def __init__(self,
                 port=None,
                 sim_step=0.1,
                 emission_path=None,
                 lateral_resolution=None,
                 no_step_log=True,
                 render=False,
                 save_render=False,
                 sight_radius=25,
                 show_radius=False,
                 pxpm=2,
                 overtake_right=False,
                 seed=None,
                 restart_instance=False,
                 print_warnings=True,
                 teleport_time=-1,
                 num_clients=1,
                 sumo_binary=None):

### 3.1 List of possible parameters

Following are the parameters which can be set up within `SumoParams`, along with their default values and description:

- `port` (_int, optional_): Port for Traci to connect to; finds an empty port by default
- `sim_step` (_float, optional_): seconds per simulation step; 0.1 by default
- `emission_path` (_str, optional_): Path to the folder in which to create the emissions output. Emissions output is not generated if this value is not specified
- `lateral_resolution` (_float, optional_): width of the divided sublanes within a lane, defaults to None (i.e. no sublanes). If this value is specified, the vehicle in the network cannot use the "LC2013" lane change model.
- `no_step_log` (_bool, optional_): specifies whether to add SUMO's step logs to the log file, and print them into the terminal during runtime, defaults to True
- `render` (_str or bool, optional_): specifies whether to visualize the rollout(s)
    - False: no rendering
    - True: delegate rendering to SUMO-GUI for back-compatibility
    - "gray": static grayscale rendering, which is good for training
    - "dgray": dynamic grayscale rendering
    - "rgb": static RGB rendering
    - "drgb": dynamic RGB rendering, which is good for visualization
- `save_render` (_bool, optional_): specifies whether to save rendering data to disk
- `sight_radius` (_int, optional_): sets the radius of observation for RL vehicles (meter)
- `show_radius` (_bool, optional_): specifies whether to render the radius of RL observation
- `pxpm` (_int, optional_): specifies rendering resolution (pixel / meter)
- `overtake_right` (_bool, optional_): whether vehicles are allowed to overtake on the right as well as the left
- `seed` (_int, optional_): seed for sumo instance
- `restart_instance` (_bool, optional): specifies whether to restart a sumo instance upon reset. Restarting the instance helps avoid slowdowns cause by excessive inflows over large experiment runtimes, but also require the gui to be started after every reset if "render" is set to True.
- `print_warnings` (_bool, optional_): If set to false, this will silence sumo warnings on the stdout
- `teleport_time` (_int, optional_): If negative, vehicles don't teleport in gridlock. If positive, they teleport after teleport_time seconds
- `num_clients` (_int, optional_): Number of clients that will connect to Traci

### 3.2 Example uing the Ring experiment

Similar to the two previous sections, we'll use the ring experiment as an example.

In [5]:
# This part initializes the Ring experiment. 

from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    InitialConfig, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS
    
name = "ring_example"

vehicles = VehicleParams()
    
vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)
    

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
initial_config = InitialConfig()


Below is the cell which contains SumoParams. And this is where you can perform some modifications involving this class.

In [6]:
sim_params = SumoParams(sim_step=0.1, render=True)

# sim_step = 0.1 sets the simulation time step to 0.1
# render = True lets you see the simulation using the SUMO GUI

# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 438.98462690001725
Average, std return: 438.98462690001725, 0.0
Average, std speed: 2.9266271862486435, 0.0


{'returns': [438.98462690001725],
 'velocities': [array([0.08577842, 0.17039769, 0.2538258 , ..., 2.9723844 , 2.97238436,
         2.97238432])],
 'mean_returns': [0.2926564179333444],
 'per_step_returns': [[0.008577841752034304,
   0.01703976566300353,
   0.025382572549459154,
   0.033603306012689835,
   0.041699252949377284,
   0.05014762423043607,
   0.05842187387272349,
   0.06611145348018753,
   0.0736674828825087,
   0.08108846436721634,
   0.08837313152369042,
   0.09552044441662831,
   0.1025295840672158,
   0.1093999462936479,
   0.11613113496788281,
   0.12272295475022006,
   0.12917540336729352,
   0.13548866350213284,
   0.14166309436691366,
   0.14769922302979357,
   0.1535977355667045,
   0.15935946810718535,
   0.16498539784030591,
   0.17047663404258478,
   0.17583440918468088,
   0.181060070167756,
   0.1861550697339544,
   0.19112095808870133,
   0.19595937476567316,
   0.20067204075860762,
   0.20526075093775867,
   0.20972736676296208,
   0.21407380930006784,
   0.2

Observe the differences as you change or add the parameters declared within `sim_step=SumoParams()` in cell `In [6]`.

For example, instead of using 

`sim_params = SumoParams(sim_step=0.1, render=True)`
     
try changing it to 
     
     `sim_params = SumoParams(
            sim_step=0.1,
            render=True,
            emission_path="./data/"
            )`

There should now be emission files (CSV & XML files) created within the specified location (here, it should be at the `data` folder in Flow. The XML file is the ones that contains emission information based on the built-in SUMO emission model. If you did not add `emission_path="./data/`, no such files will be created.

You can add more parameters and specify the attributes within `SumoParams` to further modify the simulation.

## 4. SumoCarFollowingParams

This section focuses on the discussion of parameters for SUMO-controlled acceleration behavior.
   
Following is the class declaration and the corresponding default values for the parameters:

            def __init__(
            self,
            speed_mode='right_of_way',
            accel=1.0,
            decel=1.5,
            sigma=0.5,
            tau=1.0,
            min_gap=2.5,
            max_speed=30,
            speed_factor=1.0,
            speed_dev=0.1,
            impatience=0.5,
            car_follow_model="IDM",
            **kwargs):

        
### 4.1 List of possible parameters

The parameters within this class are:

- speed_mode (_str or int, optional_) may be one of the following:
    - "right_of_way" (default): respect safe speed, right of way and brake hard at red lights if needed. DOES NOT respect max acceleration and deceleration which enables emergency stopping.

        Necessary to prevent custom models from crashing:
        
- "obey_safe_speed": prevents vehicles from colliding longitudinally, but can fail in cases where vehicles are allowed to lane change
- "no_collide": Human and RL cars are preventing from reaching speeds that may cause crashes (also serves as a failsafe). Note: this may lead to collisions in complex networks
- "aggressive": Human and RL cars are not limited by sumo with regard to their accelerations, and can crash longitudinally
- "all_checks": all sumo safety checks are activated
- int values may be used to define custom speed mode for the given vehicles, specified at: http://sumo.dlr.de/wiki/TraCI/Change_Vehicle_State#speed_mode_.280xb3.29

- `accel` (_float_): see Note
- `decel` (_float_): see Note
- `sigma` (_float_): see Note
- `tau` (_float_): see Note
- `min_gap` (_float_): see minGap Note
- `max_speed` (_float_): see maxSpeed Note
- `speed_factor` (_float_): see speedFactor Note
- `speed_dev` (_float_): see speedDev in Note
- `impatience` (_float_): see Note
- `car_follow_model` (_str_): see carFollowModel in Note
- `kwargs` (_dict_): used to handle deprecations

Note:
    For a description of all params, see: http://sumo.dlr.de/wiki/Definition_of_Vehicles,_Vehicle_Types,_and_Routes
    More details available at: http://sumo.dlr.de/wiki/TraCI/Change_Vehicle_State#speed_mode_.280xb3.29

### 4.2 Example using the Ring experiment

We again run the Ring experiment, but now with focus on this class.
 
Similar to the first two sections of this tutorial, the following lines of codes call the necessary functions defined in Flow. This also adds vehicles, sets up the environment, and declares network parameters.

In [7]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    SumoCarFollowingParams, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "ring_example"
sim_params = SumoParams(sim_step=0.1, render=True)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)

The SUMO Car Following Parameters class can be used within the declaration of the vehicles in the simulation.

Look for `car_following_params=SumoCarFollowingParams (speed_mode="no_collide")` in the code below and that's where you can modify this within `SumoCarFollowingParams()` to make it work the way you like it. 

Run the following code to see for yourself that it is running until the cell where you should be able to run the simulation in the SUMO GUI.

In [8]:
vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams
            (speed_mode="no_collide"),
        num_vehicles=22)

The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

In [9]:
# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 438.98462690001725
Average, std return: 438.98462690001725, 0.0
Average, std speed: 2.9266271862486435, 0.0


{'returns': [438.98462690001725],
 'velocities': [array([0.08577842, 0.17039769, 0.2538258 , ..., 2.9723844 , 2.97238436,
         2.97238432])],
 'mean_returns': [0.2926564179333444],
 'per_step_returns': [[0.008577841752034304,
   0.01703976566300353,
   0.025382572549459154,
   0.033603306012689835,
   0.041699252949377284,
   0.05014762423043607,
   0.05842187387272349,
   0.06611145348018753,
   0.0736674828825087,
   0.08108846436721634,
   0.08837313152369042,
   0.09552044441662831,
   0.1025295840672158,
   0.1093999462936479,
   0.11613113496788281,
   0.12272295475022006,
   0.12917540336729352,
   0.13548866350213284,
   0.14166309436691366,
   0.14769922302979357,
   0.1535977355667045,
   0.15935946810718535,
   0.16498539784030591,
   0.17047663404258478,
   0.17583440918468088,
   0.181060070167756,
   0.1861550697339544,
   0.19112095808870133,
   0.19595937476567316,
   0.20067204075860762,
   0.20526075093775867,
   0.20972736676296208,
   0.21407380930006784,
   0.2

Observe the differences as you change or add the parameters declared within `SumoCarFollowingParams` in cell `In [8]`.

For example, instead of using `car_following_params=SumoCarFollowingParams(speed_mode="no_collide")`
     
try changing it to 
     
     `car_following_params=SumoCarFollowingParams(
            minGap=0.01,
            tau=0.5,
            speed_mode="aggressive"
            )`

Many things can be done within this class to conform to your needs, adjust this based on your preference/needs.

This is the section where you should be able to modify the parameters within the `SumoParams` class.

The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

## 5. EnvParams

The `EnvParams` class enables you to specify the bounds of the action space and relevant coefficients to the reward function, as well as specifying how the positions of vehicles are modified in between rollouts.

Below is the definition of the class with the parameters and their corresponding default values:

    def __init__(self,
                 additional_params=None,
                 horizon=500,
                 warmup_steps=0,
                 sims_per_step=1,
                 evaluate=False):
                 
        self.additional_params = \
            additional_params if additional_params is not None else {}
        self.horizon = horizon
        self.warmup_steps = warmup_steps
        self.sims_per_step = sims_per_step
        self.evaluate = evaluate

        def get_additional_param(self, key):
        """Return a variable from additional_params."""
        return self.additional_params[key]

### 5.1 List of possible parameters

Following are the parameters which can be set up within `EnvParams`, along with their default values and description:

- `additional_params` (_dict, optional_): Specify additional environment params for a specific environment configuration
- `horizon` (_int, optional_)
- `warmup_steps` (_int, optional_): number of steps per rollouts; this is the number of steps performed before the initialization of training during a rollout. These warmup steps are not added as steps into training, and the actions of RL agents during these steps are dictated by SUMO. Defaults to zero
- `sims_per_step` (_int, optional_): number of SUMO simulation steps performed in any given rollout step. RL agents perform the same action for the duration of these simulation steps.
- `evaluate` (_bool, optional_): flag indicating that the evaluation reward should be used so the evaluation reward should be used rather than the normal reward  

### 5.2 Example using the Ring experiment

Similar to that of the previous sections, we'll use the Ring experiment as an example.

In [10]:
# This part initializes the Ring experiment. 

from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    InitialConfig, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "ring_example"

vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        num_vehicles=22)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
initial_config = InitialConfig()

sim_params = SumoParams(sim_step=0.1, render=True)

This is the section where you should be able to modify the parameters within the `EnvParams` class.

In [11]:
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

### The cell below, when ran, completes the scenario and should be able to render the simulation in the SUMO GUI.

In [12]:
# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 438.98462690001725
Average, std return: 438.98462690001725, 0.0
Average, std speed: 2.9266271862486435, 0.0


{'returns': [438.98462690001725],
 'velocities': [array([0.08577842, 0.17039769, 0.2538258 , ..., 2.9723844 , 2.97238436,
         2.97238432])],
 'mean_returns': [0.2926564179333444],
 'per_step_returns': [[0.008577841752034304,
   0.01703976566300353,
   0.025382572549459154,
   0.033603306012689835,
   0.041699252949377284,
   0.05014762423043607,
   0.05842187387272349,
   0.06611145348018753,
   0.0736674828825087,
   0.08108846436721634,
   0.08837313152369042,
   0.09552044441662831,
   0.1025295840672158,
   0.1093999462936479,
   0.11613113496788281,
   0.12272295475022006,
   0.12917540336729352,
   0.13548866350213284,
   0.14166309436691366,
   0.14769922302979357,
   0.1535977355667045,
   0.15935946810718535,
   0.16498539784030591,
   0.17047663404258478,
   0.17583440918468088,
   0.181060070167756,
   0.1861550697339544,
   0.19112095808870133,
   0.19595937476567316,
   0.20067204075860762,
   0.20526075093775867,
   0.20972736676296208,
   0.21407380930006784,
   0.2

Observe the differences as you change or add the parameters declared within `EnvParams` in cell `In [11]`.

For example, instead of using 

`env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)`
     
try changing it to: 
     
     `env_params = EnvParams(
            warmup_steps=40,
            sims_per_step=1,
            horizon=HORIZON,
            additional_params=additional_env_params,
            )`


You can add more parameters and specify the attributes within `EnvParams` to further modify the simulation.

## 6. VehicleParams

This is used to describe the state of all vehicles in the network. This allows you to state the information on the vehicles for a given time step can be set or retrieved from this class.

### 6.1 List of possible parameters

- `veh_id` (_str_): base vehicle ID for the vehicles (will be appended by a number)
- `acceleration_controller` (_tup, optional_):
    - 1st element: flow-specified acceleration controller
    - 2nd element: controller parameters (may be set to None to maintain default parameters)
- `lane_change_controller` (_tup, optional_):
    - 1st element: flow-specified lane-changer controller
    - 2nd element: controller parameters (may be set to None to maintain default parameters)
- `routing_controller` (_tup, optional_):
    - 1st element: flow-specified routing controller
    - 2nd element: controller parameters (may be set to None to maintain default parameters)
- `initial_speed` (_float, optional_): initial speed of the vehicles being added (in m/s)
- `num_vehicles` (_int, optional_): number of vehicles of this type to be added to the network
- `car_following_params` (from _flow.core.params.SumoCarFollowingParams_): Params object specifying attributes for SUMO car following model.
- `lane_change_params` (from _flow.core.params.SumoLaneChangeParams_): Params object specifying attributes for Sumo lane changing model.

### 6.2 Example using the Ring experiment

The following lines of codes create a basic experiment in a ring as described in tutorial 1. This one focuses on how you can apply changes in `VehicleParams`. The parameters above can be added/modified inside the code `vehicles.add()` as can be observed in the example below. You can try modifying any of the parameters in `vehicles.add()` according to the values that you need in your simulation involving the properties of vehicle to be added in the system at the start of the simulation.

In [13]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    SumoCarFollowingParams, NetParams, VehicleParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "ring_example"
sim_params = SumoParams(sim_step=0.1, render=True)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params=additional_net_params)
#initial_config = InitialConfig(bunching=20)

vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams
            (speed_mode="no_collide"),
        num_vehicles=22)

# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 438.98462690001725
Average, std return: 438.98462690001725, 0.0
Average, std speed: 2.9266271862486435, 0.0


{'returns': [438.98462690001725],
 'velocities': [array([0.08577842, 0.17039769, 0.2538258 , ..., 2.9723844 , 2.97238436,
         2.97238432])],
 'mean_returns': [0.2926564179333444],
 'per_step_returns': [[0.008577841752034304,
   0.01703976566300353,
   0.025382572549459154,
   0.033603306012689835,
   0.041699252949377284,
   0.05014762423043607,
   0.05842187387272349,
   0.06611145348018753,
   0.0736674828825087,
   0.08108846436721634,
   0.08837313152369042,
   0.09552044441662831,
   0.1025295840672158,
   0.1093999462936479,
   0.11613113496788281,
   0.12272295475022006,
   0.12917540336729352,
   0.13548866350213284,
   0.14166309436691366,
   0.14769922302979357,
   0.1535977355667045,
   0.15935946810718535,
   0.16498539784030591,
   0.17047663404258478,
   0.17583440918468088,
   0.181060070167756,
   0.1861550697339544,
   0.19112095808870133,
   0.19595937476567316,
   0.20067204075860762,
   0.20526075093775867,
   0.20972736676296208,
   0.21407380930006784,
   0.2

## 7. SumoLaneChangeParams

This is used to describe the SUMO-controlled lane change behavior.

### 7.1 List of possible parameters


- `lane_change_mode` (_str or int, optional_): may be one of the following:

    - "no_lat_collide" (default): Human cars will not make lane changes, RL cars can lane change into any space, no matter how likely it is to crash
    - "strategic": Human cars make lane changes in accordance with SUMO to provide speed boosts
    - "aggressive": RL cars are not limited by sumo with regard to their lane-change actions, and can crash longitudinally
    - * int values may be used to define custom lane change modes for the given vehicles, specified at: http://sumo.dlr.de/wiki/TraCI/Change_Vehicle_State#lane_change_mode_.280xb6.29

- `model` (_str, optional_): see laneChangeModel in Note
- `lc_strategic` (_float, optional_): see lcStrategic in Note
- `lc_cooperative` (_float, optional_): see lcCooperative in Note
- `lc_speed_gain` (_float, optional_): see lcSpeedGain in Note
- `lc_keep_right` (_float, optional_): see lcKeepRight in Note
- `lc_look_ahead_left` (_float, optional_): see lcLookaheadLeft in Note
- `lc_speed_gain_right` (_float, optional_): see lcSpeedGainRight in Note
- `lc_sublane` (_float, optional_): see lcSublane in Note
- `lc_pushy` (_float, optional_): see lcPushy in Note
- `lc_pushy_gap` (_float, optional_): see lcPushyGap in Note
- `lc_assertive` (_float, optional_): see lcAssertive in Note
- `lc_impatience` (_float, optional_): see lcImpatience in Note
- `lc_time_to_impatience` (_float, optional_): see lcTimeToImpatience in Note
- `lc_accel_lat` (_float, optional_): see lcAccelLate in Note
- `kwargs` (_dict_): used to handle deprecations


Note:
    For a description of all params, see:
    http://sumo.dlr.de/wiki/Definition_of_Vehicles,_Vehicle_Types,_and_Routes


### 7.2 Example using the Ring experiment with two lanes 

Same to that of the previous sections, we'll use the Ring experiment as an example but with __two lane__ now to illustrate how the SumoLaneChangeParams work.


In [14]:
from flow.controllers import IDMController, ContinuousRouter
from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, \
    SumoCarFollowingParams, NetParams, VehicleParams, SumoLaneChangeParams
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.scenarios.loop import LoopScenario, ADDITIONAL_NET_PARAMS

name = "ring_example"

sim_params = SumoParams(sim_step=0.1, render=True)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

additional_net_params = ADDITIONAL_NET_PARAMS.copy()
net_params = NetParams(additional_params={"length":230, "lanes": 2,
                                         "speed_limit": 30, "resolution": 40})

#initial_config = InitialConfig(bunching=20)

vehicles = VehicleParams()

vehicles.add("human",
        acceleration_controller=(IDMController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams
            (speed_mode="no_collide"),
        num_vehicles=22,
        lane_change_params=SumoLaneChangeParams(lane_change_mode=1621))

# create the scenario object
scenario = LoopScenario(
    name="ring",
    vehicles=vehicles,
    net_params=net_params)

# create the environment object
env = AccelEnv(env_params, sim_params, scenario)

exp = Experiment(env)

# run for a set number of rollouts / time steps
exp.run(1, 1500)

Round 0, return: 1035.3057591746283
Average, std return: 1035.3057591746283, 0.0
Average, std speed: 12.312173997224944, 0.0


{'returns': [1035.3057591746283],
 'velocities': [array([ 0.09745553,  0.19475589,  0.29189395, ..., 13.10497969,
         13.10497961, 13.10497953])],
 'mean_returns': [0.6902038394497522],
 'per_step_returns': [[0.009745552834899345,
   0.01947558843852297,
   0.029189395143984772,
   0.03888626603604982,
   0.04856549912586235,
   0.05822639752312444,
   0.06786826960565596,
   0.07749042918626196,
   0.08709219567684477,
   0.0966728942496899,
   0.10623185599586171,
   0.11576841808064846,
   0.1252819238959911,
   0.13477172320984077,
   0.14423717231238398,
   0.15367763415908203,
   0.16309247851047068,
   0.1724810820686673,
   0.18184282861053797,
   0.19117710911747512,
   0.20048332190174145,
   0.20976087272933935,
   0.2190091749393588,
   0.2282276495597738,
   0.23741572541964318,
   0.24657283925768833,
   0.25569843582721274,
   0.2647919679973351,
   0.27385289685051234,
   0.28288069177632313,
   0.2918748305614978,
   0.30083479947616937,
   0.309760093356332,
   0

Look for `lane_change_params=SumoLaneChangeParams(lane_change_mode=1621)` in the code above and try substituing other values to `lane_change_mode` or substituting other parameters to it. Again, refer to the given note to know more about the parameters.